# Custom Model
This guide will step through the process of implementing a custom model that can be used in NeuRec. It will show example code snippets that can help build a custom model, however, further development is needed to successfully complete the model.

The process of implementing a custom model involves two main steps: creating a model class; and, adding the model to NeuRec. The following sections discusses these two steps in detail.

## Creating a Model Class
This step involves creating a class in the **model** folder. This class must extend the **AbstractRecommender** class, also in the model folder, and implement the required functions, namely:

* \_\_init\_\_()
* build_graph()
* train_model()
* predict()

> The class may have additional functions on top of these required functions.

An example of a shell custom model is shown below:

```python
from neurec.model.AbstractRecommender import AbstractRecommender

class Custom(AbstractRecommender):
    def __init__(self):
        none
        
    def build_graph(self):
        none
        
    def train_model(self):
        none
    
    def predict(self):
        none
```

The following sections will discuss each of the function in turn, starting with creating configuration settings.

### Model Configuration
You may want some configuration settings for your custom model. This can be achieved by creating a file in the **conf** folder, for example Custom.properties. The contents of this file may look like the following:

```
[hyperparameters]
epochs=100
batch_size=256
layers=[64,32,16,8]
reg_mlp=0.0
topK=10
learning_rate=0.001
learner=adam
ispairwise=false
num_neg=4
#pairwise:BPR,hinge,square
#pointwise:cross_entropy,square
loss_function=cross_entropy
verbose=1
```

### \_\_init\_\_()
This function can be used to initialise any settings that are required by the model. This includes any configuration settings, discussed above, which can be read into the model like so:

```python
from neurec.evaluation import Evaluate
from neurec.model.AbstractRecommender import AbstractRecommender
from neurec.util import reader, learner
import numpy as np
import tensorflow as tf

class Custom(AbstractRecommender):
    def __init__(self):
        self.conf = reader.config("Custom.properties", "hyperparameters")
        
        self.layers = list(eval(self.conf["layers"]))
        self.learning_rate = float(self.conf["learning_rate"])
        self.learner = self.conf["learner"]
        self.loss_function = self.conf["loss_function"]
        self.num_epochs= int(self.conf["epochs"])
        self.batch_size= int(self.conf["batch_size"])
        # etc...
        
    # ...
```

### build_graph()
The build_graph() function is called by NeuRec before training the model and, therefore, can be used to create the network. Here, you can setup the loss and optimiser functions for the model, which can then be used while training the model:

```python
from neurec.evaluation import Evaluate
from neurec.model.AbstractRecommender import AbstractRecommender
from neurec.util import reader, learner
import numpy as np
import tensorflow as tf

class Custom(AbstractRecommender):
    # ...
    
    def build_graph(self):
        self.loss = learner.pairwise_loss(self.loss_function,result) + self.reg_mf * ( tf.reduce_sum(tf.square(p1)) \
                + tf.reduce_sum(tf.square(q2)) + tf.reduce_sum(tf.square(q1)))
        
        self.optimizer = learner.optimizer(self.learner, self.loss, self.learning_rate)
        
    # ...
```

### train_model()
This function is called by NeuRec to train the model. You can use the data_gen module to generate the necessary data for training, which can be found in the **util** folder. Additionally, the Evaluate class can be used here to establish the performance of the model, in this example after every epoch.

```python
from neurec.evaluation import Evaluate
from neurec.model.AbstractRecommender import AbstractRecommender
from neurec.util import reader, learner
import numpy as np
import tensorflow as tf

class Custom(AbstractRecommender):
    # ...
    
    def train_model(self):
        for epoch in range(self.num_epochs):
            user_input, item_input_pos, item_input_neg = data_gen._get_pairwise_all_data(self.dataset)
            
            num_training_instances = len(user_input)
            
            for num_batch in np.arange(int(num_training_instances/self.batch_size)):
                bat_users, bat_items_pos, bat_items_neg = data_gen._get_pairwise_batch_data(user_input, item_input_pos, item_input_neg, num_batch, self.batch_size)
                
            Evaluate.test_model(self,self.dataset)
            
    # ...
```

### predict()
The predict() function is used by the Evaluate classes to establish the performance of the model. An example of a predict function is given below:

```python
from neurec.evaluation import Evaluate
from neurec.model.AbstractRecommender import AbstractRecommender
from neurec.util import reader, learner
import numpy as np
import tensorflow as tf

class Custom(AbstractRecommender):
    # ...
    
    def predict(self):
        users = np.full(len(items), user_id, dtype=np.int32)
        return self.sess.run(self.output, feed_dict={self.user_input: users, self.item_input: items})
    
    # ...
```

## Adding to NeuRec
After creating a custom model, NeuRec needs to be updated so that it knows the model is available. To do this, the model needs to be added to **neurec.py** as follows:

```python
from neurec.model.Custom import Custom

def run():
    """Trains and evaluates a model."""
    with tf.compat.v1.Session(config=config) as sess:
        if recommender.lower() == "mf" :
            model = MF(sess,dataset)
            
        # ...
            
        elif recommender.lower() == "custom":
            model = Custom(sess,dataset)
```

#### Properties
Additionally, the settings for NeuRec need to be adjusted so that it uses the model Custom. Change the **recommender** value in **conf/neurec.properties** to custom, like below:

```
[default]
data.input.path=dataset/
data.input.dataset=Ciao
#UIRT
data.column.format=UIRT
#"\t" " ","::", ","
data.convert.separator=','
# binThold = -1.0 do nothing
# binThold = value, rating > value is changed to 1.0 other is 0.0.
data.convert.binarize.threshold=0
#ratio, loo, given
data.splitter=ratio
data.splitterratio=[0.8,0.2]
rec.number.thread=20
#0,99 the number of negative instances; 
#For a given user, 0 means that all the unobserved items 
#will be used to evaluate.
rec.evaluate.neg=0
recommender=custom
topK=10
```

The custom model is now ready for training and evaluating with NeuRec.

> See 2. Get Started to find out to run NeuRec.